In [ ]:
from glob import glob
import sys, time, os, asyncio
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import numpy as np
import logging

from lsst_efd_client import EfdClient
from lsst.summit.utils.efdUtils import makeEfdClient, calcNextDay, getEfdData
from lsst.summit.utils.tmaUtils import TMAEventMaker, TMAState
import lsst.sitcom.vandv.m1m3.sitcomtn81.sitcomtn_81_identify_oscillations as sitcomtn81


summit_utils_logger = logging.getLogger("lsst.summit.utils")
summit_utils_logger.setLevel(logging.ERROR)

%matplotlib inline
%load_ext lab_black
%load_ext autoreload
%autoreload 2

# This runs the script to identify times during tma slews with oscillations in the measured hardpoints

1. Identify days that contain slews
2. For each day with a slew run the identification script
3. For each day create diagnostic plot to visually inspect identified events
4. Save visually inspected events
5. Create Chronograph links for events

## 1. identify days with slews 

In [ ]:
# check for days that have no eventMaker problems and have slews
day_start = 20231204
day_end = 20240115
day_range = []
eventMaker = TMAEventMaker()
client = eventMaker.client


while day_start <= day_end:
    day_obs = day_start
    try:
        events = eventMaker.getEvents(day_obs)
    except RuntimeError as error:
        print(f"{day_obs} Caught an error: {error}")
        day_start = calcNextDay(day_start)
        continue

    slews = [e for e in events if e.type == TMAState.SLEWING]
    try:
        print(f"{day_obs} - {len(slews)} slews")
        if len(slews) > 0:
            day_range.append(day_start)
    except:
        print(f"problem with {day_obs}")

    day_start = calcNextDay(day_start)

## 2. for each day with a slew run the identification script
this script is `sitcomtn81.IdentifyOscillationEvents`

In [ ]:
datapath = "./sitcomtn81_data"
force = False
efd_instance = "usdf_efd"
if not os.path.exists(datapath):
    os.makedirs(datapath)

id_oscillations = sitcomtn81.IdentifyOscillationEvents(efd_instance=efd_instance)

for current_day_obs in day_range:
    print(current_day_obs)
    save_string = f"{datapath}/oscillation_events_{current_day_obs}.csv"
    if os.path.exists(save_string):
        print(f"file exists: {save_string}")
        continue
    oscillation_events_frame = await id_oscillations.run(current_day_obs)
    if oscillation_events_frame is not None:
        oscillation_events_frame.to_csv(save_string)
        print("finished")
    else:
        print("none identified")



In [ ]:
def add_timestamp(data):
    """
    Adds a correct timestamp column in UTC format to the provided data if
    not present.

    Parameters
    ----------
    data : DataFrame
        The data to which the timestamp will be added.

    Returns
    -------
    DataFrame
        The data with the added 'snd_timestamp_utc' column.
    """
    if "snd_timestamp_utc" not in data.columns:
        data["snd_timestamp_utc"] = Time(
            data["private_sndStamp"], format="unix_tai"
        ).unix
    return data


def get_data(events_frame, client, train=False):
    table_dict = {
        "m1m3_hp_actuator": "lsst.sal.MTM1M3.hardpointActuatorData",
    }
    query_dict = {}
    for key in table_dict.keys():
        query_dict[key] = []
    for j, time in tqdm(enumerate(events_frame["times"])):
        event_time = Time(events_frame["times"][j] - 2, format="unix")

        for key in table_dict.keys():
            hpcols = ["private_sndStamp"] + [f"measuredForce2"]  # for i in range(6)]
            query = getEfdData(
                client=client,
                topic=table_dict[key],
                columns=hpcols,
                begin=event_time,
                end=event_time,
                prePadding=60,
                postPadding=60,
            )
            query["event_num"] = j
            query["seqNum"] = events_frame["seq_num"][j]

            query = add_timestamp(query)

            query_dict[key].append(query)
    
    # Iterate over the remaining DataFrames and merge them
    for key in sorted_keys[1:]:
        merged_df = pd.merge_asof(
            merged_df,
            query_dict[key].sort_index(),
            left_on="snd_timestamp_utc",
            right_on="snd_timestamp_utc",
            tolerance=tolerance,
            direction="nearest",
            suffixes=("", "_" + key),
        )
    merged_df.reset_index(drop=True, inplace=True)

    return merged_df

## 3. For each day create diagnostic plot to visually inspect identified events

In [ ]:
client = EfdClient("usdf_efd")
from glob import glob

flist = glob("./sitcomtn81_data/oscillation_events_*.csv")
days_with_events = []
for file in flist:
    dat = pd.read_csv(file)
    if len(dat) > 0:
        days_with_events.append(file[-12:-4])
days_with_events = sorted(days_with_events)
print(f"{len(days_with_events)} days with an event")

In [ ]:
days_with_events = [i for i in days_with_events if float(i) > 20231204]

In [ ]:
for i, j in enumerate(days_with_events):
    print(i, j)

In [ ]:
# select day from above list starting with 0 then run next 5 cells then increment this index
index = 17
day = days_with_events[index]
eventMaker = TMAEventMaker()
events = eventMaker.getEvents(int(day))

events_frame = pd.read_csv(f"./sitcomtn81_data/oscillation_events_{day}.csv")
print(indexer, day, len(events_frame))
if len(events_frame) > 1000:
    sep_sel = [True] + list((np.diff(events_frame["times"]) > 8))
    print(len(events_frame[sep_sel]))
    events_frame = events_frame[sep_sel].reset_index()
merged_df = get_data(events_frame, client)

In [ ]:
for m in range(int(len(events_frame) / 120) + 1):
    l = 4  # should be 1 or 4
    plt.figure(dpi=175, figsize=(12, 8 * l))
    j = 0
    k = 30 * l * m
    for i in np.unique(merged_df["event_num"])[k : k + 30 * l]:
        # if i not in good:
        #     continue
        plt.title(f"{day}, {k}, {l}, {k/l/30}")
        subframe = merged_df.copy()
        subframe = subframe[subframe["event_num"] == i]
        seq_val = np.unique(subframe["seqNum"])
        if len(subframe) > 0:
            plt.plot(
                subframe["delta_time"],
                subframe["measuredForce2"] - 800 * j,
                label=f"{seq_val[0]}, {i}",
            )
            plt.annotate(xy=(-20, -800 * j + 10), text=f"{seq_val[0]}, {i}")
            j += 1

    if l == 1:
        plt.legend()
    if l == 4:
        plt.legend(fontsize=9.2, loc=7, title="seqNum, eventNum")
    plt.axvline(2, c="k", ls="dashed")
    plt.savefig(f"./sitcomtn81_data/plots/diagnostic_{day}_{k}_{l}_{int(k/l/30)}.png")
    # plt.close()

## 4. Save visually inspected events

In [ ]:
# after inspecting the above plots
cleaned_event_nums = [572]

In [ ]:
l = 1
plt.figure(dpi=175, figsize=(10, 8 * l))
j = 0
k = 30 * l * m
offset = 2000
for i in np.unique(merged_df["event_num"]):  # [k : k + 30 * l]:
    if i not in cleaned_event_nums:
        continue
    plt.title(f"{day} slews with oscillations")
    subframe = merged_df.copy()
    subframe = subframe[subframe["event_num"] == i]
    seq_val = np.unique(subframe["seqNum"])
    # block_num = np.unique(subframe["block"])
    if len(subframe) > 0:
        plt.plot(
            subframe["delta_time"],  # - offset_dict[i],
            subframe["measuredForce2"]
            - subframe["measuredForce2"][subframe["delta_time"] < 0].mean()
            - offset * j,
            label=f"{seq_val[0]}",
        )
        j += 1
if l == 1:
    plt.legend(title="slew number")
if l == 4:
    plt.legend(fontsize=9.2, loc=7)
# plt.axvline(0, c="k", ls="dashed")
plt.ylabel(f"$\Delta$ HP 2 force [N] - {offset} N offset")
plt.xlabel("Time [s]")
plt.xlim(-10, 25)
plt.axvline(0, ls="dashed", c="k")
plt.savefig(f"./sitcomtn81_data/plots/osc_hp2_{day}.png")

# plt.close()

In [ ]:
seq_list = []
for row in cleaned_event_nums:
    day_obs, seq_num, times = events_frame.loc[row, ["day_obs", "seq_num", "times"]]
    print(f"{day_obs}, {seq_num}, {row}, {times}")
    seq_list.append(row)
events_frame.loc[seq_list, ["day_obs", "seq_num", "times"]].to_csv(
    f"{datapath}/cleaned_events{day}.csv",
    index=False,
)

After visually inspecting all events, the combined cleaned csv is attached to jira ticket called 231114_cleaned_oscillations.csv

In [ ]:
clean_frame = []
for day in days_with_events:
    try:
        df = pd.read_csv(f"./sitcomtn81_data/cleaned_events{day}.csv")
        df.columns = ["day_obs", "seq_num", "time"]
        clean_frame.append()
    except:
        a = 2
osc_frame_1 = pd.read_csv(
    f"./sitcomtn81_data/231121_dynamic_testing_cleaned_oscillations.csv"
)
osc_frame_1 = osc_frame_1.drop("osc_event_num", axis=1)
clean_frame.append(osc_frame_1)

osc_frame_2 = pd.read_csv(f"./sitcomtn81_data/231114_cleaned_oscillations.csv")
osc_frame_2 = osc_frame_2.drop(" osc_event_num", axis=1)
osc_frame_2.columns = ["day_obs", "seq_num", "time"]
clean_frame.append(osc_frame_2)

oscillation_frame = (
    pd.concat(clean_frame).sort_values(by="day_obs").reset_index(drop=True)
)

oscillation_frame.to_csv(
    "./sitcomtn81_data/240327_dynamic_testing_hp_osc_events.csv", index=False
)

In [ ]:
all_frame = []
for file in flist:
    dat = pd.read_csv(file)
    all_frame.append(dat)
all_frame = pd.concat(all_frame)

## 5. Create Chronograph links for the events

In [ ]:
events_frame = (
    pd.merge(all_frame, clean_frame, left_on="times", right_on="time", how="inner")
    .drop_duplicates(subset="time")
    .reset_index(drop=True)
)
events_frame["iso_time"] = Time(events_frame["times"], format="unix").iso

In [ ]:
events_frame = events_frame.sort_values("times").reset_index(drop=True)

In [ ]:
events_frame

In [ ]:
def link_from_date(timestamp):
    begin_timestamp = (Time(timestamp, format="unix") - TimeDelta(3, format="sec")).iso
    end_timestamp = (Time(timestamp, format="unix") + TimeDelta(3, format="sec")).iso

    begin_day = begin_timestamp[:10]
    begin_hour = begin_timestamp[11:13]
    begin_minute = begin_timestamp[14:16]
    begin_second = begin_timestamp[17:]

    end_day = end_timestamp[:10]
    end_hour = end_timestamp[11:13]
    end_minute = end_timestamp[14:16]
    end_second = end_timestamp[17:]

    url = "https://summit-lsp.lsst.codes/chronograf/sources/1/dashboards/"
    url += "207?refresh=Paused&tempVars%5BDownsample%5D=Default&tempVars%5BFunction%5D="
    url += f"raw&lower={begin_day}T{begin_hour}%3A{begin_minute}%3A{begin_second}Z"
    url += f"&upper={end_day}T{end_hour}%3A{end_minute}%3A{end_second}Z"
    # url+=f"&zoomedLower={begin_day}T{begin_hour}%3A{begin_minute}%3A{begin_second}Z&zoomedUpper=2023-05-30T10%3A08%3A02.962Z"
    return url


title_str = "| event number  | time  |  dayobs | slew_num |  elevation |  elevation max velocity| azimuth max velocity | hardpoint/ims dashboard  |"
title_str += "\n|---|---|---|---|---|---|---|---|\n"
# create html table for confluence
row_str = ""
ct = 0
for j, i in enumerate((range(len(events_frame)))):
    # if events["group"][i] < 3:
    ct += 1
    row_str += f"| {ct}  | {events_frame['iso_time'][i]}  | "
    row_str += f"{events_frame['day_obs_x'][i]} | {events_frame['seq_num_x'][i]} |"
    row_str += f"{events_frame['elevation_position'][i]:0.2f}  | {events_frame['elevation_velocity'][i]:0.2f} | "
    row_str += f" {events_frame['azimuth_velocity'][i]:0.2f} "
    row_str += '| <a href="'
    row_str += f'{link_from_date(events_frame["times"][i]+3)}'
    row_str += '" target="_blank">link to dashboard</a>|\n'

In [ ]:
print(title_str + row_str)